# Simulation
## Group project
### 42180 Quantitative modelling of behaviour

------

**Group 11** <br>

------

Camille Babin <br>
Martine Grangé <br>
Mikkel Johansen <br>
Kyriakos Michailidis <br>
Jiri Spitzer

------

Import libraries and enable inline plotting

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
from collections import Counter
from re import search as Search
from IPython.display import display

In [2]:
%matplotlib inline

Disable warnings

In [3]:
warnings.filterwarnings('ignore')

Load data

In [4]:
data_path = r'.\data\Elisa_thesis_data_noPC.csv'

In [5]:
df = pd.read_csv(data_path)
df.head(2)

,VAR1,ID,CHOICE,Alt_Description_ICV1,CarClass_ICV1,CarType_ICV1,Purchaseprice_ICV1,Operationcost_ICV1,Yearlycost__ICV1,Range_ICV1,...,q16_20_resp,q16_21_resp,q16_22_resp,q16_23_resp,q16_24_resp,House_single,House_couple,House_children,House_couple_children,House_couple_nochildren
0,0,1,1,1,2,1,200000,1.11,4300,1097,...,1,5,4,2,5,1,0,0,0,0
1,1,1,3,1,2,1,185000,0.88,4300,898,...,1,5,4,2,5,1,0,0,0,0


#### Data preparation

Deleting rows with unknown choice.

In [41]:
df_mod=df[df['CHOICE']>0].copy(deep=True)
df_mod.reset_index();

Saving number of rows and columns.

In [45]:
n,m=df_mod.shape

In [42]:
av_cho=[]
for i in df_mod.index:
    av_cho.append([])
    if df_mod['CarClass_ICV1'].loc[i]>0: 
        av_cho[-1].append(df_mod['CarClass_ICV1'].loc[i])
    if df_mod['CarClass_ICV2'].loc[i]>0:
        av_cho[-1].append(df_mod['CarClass_ICV2'].loc[i])
    if df_mod['CarClass_BEV1'].loc[i]>0:
        av_cho[-1].append(df_mod['CarClass_BEV1'].loc[i]+6)
    if df_mod['CarClass_BEV2'].loc[i]>0:
        av_cho[-1].append(df_mod['CarClass_BEV2'].loc[i]+6)
    if df_mod['CarClass_PHEV1'].loc[i]>0: 
        av_cho[-1].append(df_mod['CarClass_PHEV1'].loc[i]+12)
    if df_mod['CarClass_PHEV2'].loc[i]>0:
        av_cho[-1].append(df_mod['CarClass_PHEV2'].loc[i]+12)

In [46]:
avail=['AV1','AV2','AV3','AV4','AV5','AV6','AV7','AV8','AV9','AV10','AV11','AV12','AV13','AV14','AV15','AV16','AV17','AV18']
for i in range(18):
    df_mod[avail[i]]=np.zeros(n,int)
    for j in range(n):
        if i+1 in av_cho[j]:
            df_mod[avail[i]].loc[df_mod.index[j]]=1

In [47]:
df_mod['Gender_Dummy']=np.zeros(n,int)
df_mod['Dummy_Age2']=np.zeros(n,int)
df_mod['Dummy_Age3']=np.zeros(n,int)
df_mod['Dummy_Age4']=np.zeros(n,int)
df_mod['Dummy_Age5']=np.zeros(n,int)
for j in range(n):
    df_mod['Gender_Dummy'].loc[df_mod.index[j]]=df_mod['c_Gender'].loc[df_mod.index[j]]-1
    if df_mod['c_Age_cat'].loc[df_mod.index[j]]==2:
        df_mod['Dummy_Age2'].loc[df_mod.index[j]]=1
    if df_mod['c_Age_cat'].loc[df_mod.index[j]]==3:
        df_mod['Dummy_Age3'].loc[df_mod.index[j]]=1
    if df_mod['c_Age_cat'].loc[df_mod.index[j]]==4:
        df_mod['Dummy_Age4'].loc[df_mod.index[j]]=1
    if df_mod['c_Age_cat'].loc[df_mod.index[j]]==5:
        df_mod['Dummy_Age5'].loc[df_mod.index[j]]=1

In [48]:
columns=["Size_ICV_MINI","Size_ICV_LIL","Size_ICV_MEL","Size_ICV_STOR","Size_ICV_PREM","Size_ICV_LUK",
         "Size_BEV_MINI","Size_BEV_LIL","Size_BEV_MEL","Size_BEV_STOR","Size_BEV_PREM","Size_BEV_LUK",
         "Size_PHEV_MINI","Size_PHEV_LIL","Size_PHEV_MEL","Size_PHEV_STOR","Size_PHEV_PREM","Size_PHEV_LUK"]
for column in columns:
    df_mod[column+"_Large_Dummy"]=np.zeros(n,int)
    for j in range(n):
        if df_mod[column].loc[df_mod.index[j]]>2:
            df_mod[column+"_Large_Dummy"].loc[df_mod.index[j]]=1
        else:
            df_mod[column+"_Large_Dummy"].loc[df_mod.index[j]]=0

In [49]:
pd.options.display.max_columns = 500

In [50]:
df_mod.head(5)

,VAR1,ID,CHOICE,Alt_Description_ICV1,CarClass_ICV1,CarType_ICV1,Purchaseprice_ICV1,Operationcost_ICV1,Yearlycost__ICV1,Range_ICV1,Acceleration_ICV1,Size_ICV1,CO2_ICV1,Charginginfrastructure_ICV1,Chargingspeed_ICV1,Homedistnearestcharger_ICV1,Homechargeravailability_ICV1,Alt_Description_ICV2,CarClass_ICV2,CarType_ICV2,Purchaseprice_ICV2,Operationcost_ICV2,Yearlycost__ICV2,Range_ICV2,Acceleration_ICV2,Size_ICV2,CO2_ICV2,Charginginfrastructure_ICV2,Chargingspeed_ICV2,Homedistnearestcharger_ICV2,Homechargeravailability_ICV2,Alt_Description_BEV1,CarClass_BEV1,CarType_BEV1,Purchaseprice_BEV1,Operationcost_BEV1,Yearlycost__BEV1,Range_BEV1,Acceleration_BEV1,Size_BEV1,CO2_BEV1,Charginginfrastructure_BEV1,Chargingspeed_BEV1,Homedistnearestcharger_BEV1,Homechargeravailability_BEV1,Alt_Description_BEV2,CarClass_BEV2,CarType_BEV2,Purchaseprice_BEV2,Operationcost_BEV2,Yearlycost__BEV2,Range_BEV2,Acceleration_BEV2,Size_BEV2,CO2_BEV2,Charginginfrastructure_BEV2,Chargingspeed_BEV2,Homedistnearestcharger_BEV2,Homechargeravailability_BEV2,Alt_Description_PHEV1,CarClass_PHEV1,CarType_PHEV1,Purchaseprice_PHEV1,Operationcost_PHEV1,Yearlycost__PHEV1,Range_PHEV1,Acceleration_PHEV1,Size_PHEV1,CO2_PHEV1,Charginginfrastructure_PHEV1,Chargingspeed_PHEV1,Homedistnearestcharger_PHEV1,Homechargeravailability_PHEV1,Alt_Description_PHEV2,CarClass_PHEV2,CarType_PHEV2,Purchaseprice_PHEV2,Operationcost_PHEV2,Yearlycost__PHEV2,Range_PHEV2,Acceleration_PHEV2,Size_PHEV2,CO2_PHEV2,Charginginfrastructure_PHEV2,Chargingspeed_PHEV2,Homedistnearestcharger_PHEV2,Homechargeravailability_PHEV2,Purchaseprice_ICV_MINI,Operationcost_ICV_MINI,Yearlycost__ICV_MINI,Range_ICV_MINI,Acceleration_ICV_MINI,Size_ICV_MINI,CO2_ICV_MINI,Charginginfrastructure_ICV_MINI,Chargingspeed_ICV_MINI,Homedistnearestcharger_ICV_MINI,Homechargeravailability_ICV_MINI,Purchaseprice_ICV_LIL,Operationcost_ICV_LIL,Yearlycost__ICV_LIL,Range_ICV_LIL,Acceleration_ICV_LIL,Size_ICV_LIL,CO2_ICV_LIL,Charginginfrastructure_ICV_LIL,Chargingspeed_ICV_LIL,Homedistnearestcharger_ICV_LIL,Homechargeravailability_ICV_LIL,Purchaseprice_ICV_MEL,Operationcost_ICV_MEL,Yearlycost__ICV_MEL,Range_ICV_MEL,Acceleration_ICV_MEL,Size_ICV_MEL,CO2_ICV_MEL,Charginginfrastructure_ICV_MEL,Chargingspeed_ICV_MEL,Homedistnearestcharger_ICV_MEL,Homechargeravailability_ICV_MEL,Purchaseprice_ICV_STOR,Operationcost_ICV_STOR,Yearlycost__ICV_STOR,Range_ICV_STOR,Acceleration_ICV_STOR,Size_ICV_STOR,CO2_ICV_STOR,Charginginfrastructure_ICV_STOR,Chargingspeed_ICV_STOR,Homedistnearestcharger_ICV_STOR,Homechargeravailability_ICV_STOR,Purchaseprice_ICV_PREM,Operationcost_ICV_PREM,Yearlycost__ICV_PREM,Range_ICV_PREM,Acceleration_ICV_PREM,Size_ICV_PREM,CO2_ICV_PREM,Charginginfrastructure_ICV_PREM,Chargingspeed_ICV_PREM,Homedistnearestcharger_ICV_PREM,Homechargeravailability_ICV_PREM,Purchaseprice_ICV_LUK,Operationcost_ICV_LUK,Yearlycost__ICV_LUK,Range_ICV_LUK,Acceleration_ICV_LUK,Size_ICV_LUK,CO2_ICV_LUK,Charginginfrastructure_ICV_LUK,Chargingspeed_ICV_LUK,Homedistnearestcharger_ICV_LUK,Homechargeravailability_ICV_LUK,Purchaseprice_BEV_MINI,Operationcost_BEV_MINI,Yearlycost__BEV_MINI,Range_BEV_MINI,Acceleration_BEV_MINI,Size_BEV_MINI,CO2_BEV_MINI,Charginginfrastructure_BEV_MINI,Chargingspeed_BEV_MINI,Homedistnearestcharger_BEV_MINI,Homechargeravailability_BEV_MINI,Purchaseprice_BEV_LIL,Operationcost_BEV_LIL,Yearlycost__BEV_LIL,Range_BEV_LIL,Acceleration_BEV_LIL,Size_BEV_LIL,CO2_BEV_LIL,Charginginfrastructure_BEV_LIL,Chargingspeed_BEV_LIL,Homedistnearestcharger_BEV_LIL,Homechargeravailability_BEV_LIL,Purchaseprice_BEV_MEL,Operationcost_BEV_MEL,Yearlycost__BEV_MEL,Range_BEV_MEL,Acceleration_BEV_MEL,Size_BEV_MEL,CO2_BEV_MEL,Charginginfrastructure_BEV_MEL,Chargingspeed_BEV_MEL,Homedistnearestcharger_BEV_MEL,Homechargeravailability_BEV_MEL,Purchaseprice_BEV_STOR,Operationcost_BEV_STOR,Yearlycost__BEV_STOR,Range_BEV_STOR,Acceleration_BEV_STOR,Size_BEV_STOR,CO2_BEV_STOR,Charginginfrastructure_BEV_STOR,Chargingspeed_BEV_STOR,Homedistnearestcharger_BEV_STOR,Home

In [85]:
pd.options.display.max_columns = 50

#### Utility functions

In [30]:
'''
V1 = ASC_1_RND + B_PP*Purchaseprice_ICV_MINI + B_OC*Operationcost_ICV_MINI + B_YC*Yearlycost__ICV_MINI + B_Ra*Range_ICV_MINI + B_Acc*Acceleration_ICV_MINI + B_SizeLarge*Size_ICV_MINI_Large_Dummy + B_Co2*CO2_ICV_MINI + B_ChInf*Charginginfrastructure_ICV_MINI + B_ChSpeed*Chargingspeed_ICV_MINI + B_Dist*Homedistnearestcharger_ICV_MINI + B_HomeCh*Homechargeravailability_ICV_MINI
V2 = ASC_2_RND + B_PP*Purchaseprice_ICV_LIL + B_OC*Operationcost_ICV_LIL + B_YC*Yearlycost__ICV_LIL + B_Ra*Range_ICV_LIL + B_Acc*Acceleration_ICV_LIL + B_SizeLarge*Size_ICV_LIL_Large_Dummy + B_Co2*CO2_ICV_LIL + B_ChInf*Charginginfrastructure_ICV_LIL + B_ChSpeed*Chargingspeed_ICV_LIL + B_Dist*Homedistnearestcharger_ICV_LIL + B_HomeCh*Homechargeravailability_ICV_LIL
V3 = ASC_3_RND + B_PP*Purchaseprice_ICV_MEL + B_OC*Operationcost_ICV_MEL + B_YC*Yearlycost__ICV_MEL + B_Ra*Range_ICV_MEL + B_Acc*Acceleration_ICV_MEL + B_SizeLarge*Size_ICV_MEL_Large_Dummy + B_Co2*CO2_ICV_MEL + B_ChInf*Charginginfrastructure_ICV_MEL + B_ChSpeed*Chargingspeed_ICV_MEL + B_Dist*Homedistnearestcharger_ICV_MEL + B_HomeCh*Homechargeravailability_ICV_MEL
V4 = ASC_4_RND + B_PP*Purchaseprice_ICV_STOR + B_OC*Operationcost_ICV_STOR + B_YC*Yearlycost__ICV_STOR + B_Ra*Range_ICV_STOR + B_Acc*Acceleration_ICV_STOR + B_SizeLarge*Size_ICV_STOR_Large_Dummy + B_Co2*CO2_ICV_STOR + B_ChInf*Charginginfrastructure_ICV_STOR + B_ChSpeed*Chargingspeed_ICV_STOR + B_Dist*Homedistnearestcharger_ICV_STOR + B_HomeCh*Homechargeravailability_ICV_STOR
V5 = ASC_5_RND + B_PP*Purchaseprice_ICV_PREM + B_OC*Operationcost_ICV_PREM + B_YC*Yearlycost__ICV_PREM + B_Ra*Range_ICV_PREM + B_Acc*Acceleration_ICV_PREM + B_SizeLarge*Size_ICV_PREM_Large_Dummy + B_Co2*CO2_ICV_PREM + B_ChInf*Charginginfrastructure_ICV_PREM + B_ChSpeed*Chargingspeed_ICV_PREM + B_Dist*Homedistnearestcharger_ICV_PREM + B_HomeCh*Homechargeravailability_ICV_PREM
V6 = ASC_6_RND + B_PP*Purchaseprice_ICV_LUK + B_OC*Operationcost_ICV_LUK + B_YC*Yearlycost__ICV_LUK + B_Ra*Range_ICV_LUK + B_Acc*Acceleration_ICV_LUK + B_SizeLarge*Size_ICV_LUK_Large_Dummy + B_Co2*CO2_ICV_LUK + B_ChInf*Charginginfrastructure_ICV_LUK + B_ChSpeed*Chargingspeed_ICV_LUK + B_Dist*Homedistnearestcharger_ICV_LUK + B_HomeCh*Homechargeravailability_ICV_LUK

V7 = ASC_7_RND + B_PP*Purchaseprice_BEV_MINI + B_OC*Operationcost_BEV_MINI + B_YC*Yearlycost__BEV_MINI + B_Ra*Range_BEV_MINI + B_Acc*Acceleration_BEV_MINI + B_SizeLarge*Size_BEV_MINI_Large_Dummy + B_Co2*CO2_BEV_MINI + B_ChInf*Charginginfrastructure_BEV_MINI + B_ChSpeed*Chargingspeed_BEV_MINI + B_Dist*Homedistnearestcharger_BEV_MINI + B_HomeCh*Homechargeravailability_BEV_MINI
V8 = ASC_8_RND + B_PP*Purchaseprice_BEV_LIL + B_OC*Operationcost_BEV_LIL + B_YC*Yearlycost__BEV_LIL + B_Ra*Range_BEV_LIL + B_Acc*Acceleration_BEV_LIL + B_SizeLarge*Size_BEV_LIL_Large_Dummy + B_Co2*CO2_BEV_LIL + B_ChInf*Charginginfrastructure_BEV_LIL + B_ChSpeed*Chargingspeed_BEV_LIL + B_Dist*Homedistnearestcharger_BEV_LIL + B_HomeCh*Homechargeravailability_BEV_LIL
V9 = ASC_9_RND + B_PP*Purchaseprice_BEV_MEL + B_OC*Operationcost_BEV_MEL + B_YC*Yearlycost__BEV_MEL + B_Ra*Range_BEV_MEL + B_Acc*Acceleration_BEV_MEL + B_SizeLarge*Size_BEV_MEL_Large_Dummy + B_Co2*CO2_BEV_MEL + B_ChInf*Charginginfrastructure_BEV_MEL + B_ChSpeed*Chargingspeed_BEV_MEL + B_Dist*Homedistnearestcharger_BEV_MEL + B_HomeCh*Homechargeravailability_BEV_MEL
V10 = ASC_10_RND + B_PP*Purchaseprice_BEV_STOR + B_OC*Operationcost_BEV_STOR + B_YC*Yearlycost__BEV_STOR + B_Ra*Range_BEV_STOR + B_Acc*Acceleration_BEV_STOR + B_SizeLarge*Size_BEV_STOR_Large_Dummy + B_Co2*CO2_BEV_STOR + B_ChInf*Charginginfrastructure_BEV_STOR + B_ChSpeed*Chargingspeed_BEV_STOR + B_Dist*Homedistnearestcharger_BEV_STOR + B_HomeCh*Homechargeravailability_BEV_STOR
V11 = ASC_11_RND + B_PP*Purchaseprice_BEV_PREM + B_OC*Operationcost_BEV_PREM + B_YC*Yearlycost__BEV_PREM + B_Ra*Range_BEV_PREM + B_Acc*Acceleration_BEV_PREM + B_SizeLarge*Size_BEV_PREM_Large_Dummy + B_Co2*CO2_BEV_PREM + B_ChInf*Charginginfrastructure_BEV_PREM + B_ChSpeed*Chargingspeed_BEV_PREM + B_Dist*Homedistnearestcharger_BEV_PREM + B_HomeCh*Homechargeravailability_BEV_PREM
V12 = ASC_12_RND + B_PP*Purchaseprice_BEV_LUK + B_OC*Operationcost_BEV_LUK + B_YC*Yearlycost__BEV_LUK + B_Ra*Range_BEV_LUK + B_Acc*Acceleration_BEV_LUK + B_SizeLarge*Size_BEV_LUK_Large_Dummy + B_Co2*CO2_BEV_LUK + B_ChInf*Charginginfrastructure_BEV_LUK + B_ChSpeed*Chargingspeed_BEV_LUK + B_Dist*Homedistnearestcharger_BEV_LUK + B_HomeCh*Homechargeravailability_BEV_LUK

V13 = ASC_13_RND + B_PP*Purchaseprice_PHEV_MINI + B_OC*Operationcost_PHEV_MINI + B_YC*Yearlycost__PHEV_MINI + B_Ra*Range_PHEV_MINI + B_Acc*Acceleration_PHEV_MINI + B_SizeLarge*Size_PHEV_MINI_Large_Dummy + B_Co2*CO2_PHEV_MINI + B_ChInf*Charginginfrastructure_PHEV_MINI + B_ChSpeed*Chargingspeed_PHEV_MINI + B_Dist*Homedistnearestcharger_PHEV_MINI + B_HomeCh*Homechargeravailability_PHEV_MIN
V14 = ASC_14_RND + B_PP*Purchaseprice_PHEV_LIL + B_OC*Operationcost_PHEV_LIL + B_YC*Yearlycost__PHEV_LIL + B_Ra*Range_PHEV_LIL + B_Acc*Acceleration_PHEV_LIL + B_SizeLarge*Size_PHEV_LIL_Large_Dummy + B_Co2*CO2_PHEV_LIL + B_ChInf*Charginginfrastructure_PHEV_LIL + B_ChSpeed*Chargingspeed_PHEV_LIL + B_Dist*Homedistnearestcharger_PHEV_LIL + B_HomeCh*Homechargeravailability_PHEV_LIL
V15 = ASC_15_RND + B_PP*Purchaseprice_PHEV_MEL + B_OC*Operationcost_PHEV_MEL + B_YC*Yearlycost__PHEV_MEL + B_Ra*Range_PHEV_MEL + B_Acc*Acceleration_PHEV_MEL + B_SizeLarge*Size_PHEV_MEL_Large_Dummy + B_Co2*CO2_PHEV_MEL + B_ChInf*Charginginfrastructure_PHEV_MEL + B_ChSpeed*Chargingspeed_PHEV_MEL + B_Dist*Homedistnearestcharger_PHEV_MEL + B_HomeCh*Homechargeravailability_PHEV_MEL
V16 = ASC_16_RND + B_PP*Purchaseprice_PHEV_STOR + B_OC*Operationcost_PHEV_STOR + B_YC*Yearlycost__PHEV_STOR + B_Ra*Range_PHEV_STOR + B_Acc*Acceleration_PHEV_STOR + B_SizeLarge*Size_PHEV_STOR_Large_Dummy + B_Co2*CO2_PHEV_STOR + B_ChInf*Charginginfrastructure_PHEV_STOR + B_ChSpeed*Chargingspeed_PHEV_STOR + B_Dist*Homedistnearestcharger_PHEV_STOR + B_HomeCh*Homechargeravailability_PHEV_STO
V17 = ASC_17_RND + B_PP*Purchaseprice_PHEV_PREM + B_OC*Operationcost_PHEV_PREM + B_YC*Yearlycost__PHEV_PREM + B_Ra*Range_PHEV_PREM + B_Acc*Acceleration_PHEV_PREM + B_SizeLarge*Size_PHEV_PREM_Large_Dummy + B_Co2*CO2_PHEV_PREM + B_ChInf*Charginginfrastructure_PHEV_PREM + B_ChSpeed*Chargingspeed_PHEV_PREM + B_Dist*Homedistnearestcharger_PHEV_PREM + B_HomeCh*Homechargeravailability_PHEV_PRE
V18 = ASC_18 + B_PP*Purchaseprice_PHEV_LUK + B_OC*Operationcost_PHEV_LUK + B_YC*Yearlycost__PHEV_LUK + B_Ra*Range_PHEV_LUK + B_Acc*Acceleration_PHEV_LUK + B_SizeLarge*Size_PHEV_LUK_Large_Dummy + B_Co2*CO2_PHEV_LUK + B_ChInf*Charginginfrastructure_PHEV_LUK + B_ChSpeed*Chargingspeed_PHEV_LUK + B_Dist*Homedistnearestcharger_PHEV_LUK + B_HomeCh*Homechargeravailability_PHEV_LUK
''';

Parameters

In [26]:
ASC_1 = -6.17
ASC_1_S = 2.86
ASC_2 = -6.51
ASC_2_S = 3.27
ASC_3 = -5.41
ASC_3_S = 2.81
ASC_4 = -5.9
ASC_4_S = 2.86
ASC_5 = -5.67
ASC_5_S = 3.11
ASC_6 = -5.92
ASC_6_S = 5.38
ASC_7 = -5.57
ASC_7_S = 2.34
ASC_8 = -6.15
ASC_8_S = 2.04
ASC_9 = -5.01
ASC_9_S = 2.02
ASC_10 = -5.12
ASC_10_S = 2.04
ASC_11 = -5.02
ASC_11_S = 2.46
ASC_12 = -3.14
ASC_12_S = 2.37
ASC_13 = -4.7
ASC_13_S = 2.29
ASC_14 = -5.29
ASC_14_S = 2.75
ASC_15 = -3.52
ASC_15_S = 2.73
ASC_16 = -3.59
ASC_16_S = 2.32
ASC_17 = -3.88
ASC_17_S = 2.6

B_Acc = -0.0345
B_ChInf = -0.000261
B_ChSpeed = 0.0039
B_Co2 = -0.00325
B_Dist = -0.00106
B_HomeCh = 0.137
B_OC = -0.608
B_PP = -6.5 * 10 ** -6
B_Ra = 0.0019
B_SizeLarge = 0.23
B_YC = -9.55 * 10 ** -5

Utility functions ICEV

In [54]:
def V1(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V2(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V3(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V4(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V5(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V6(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

Utility functions BEV

In [70]:
def V7(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V8(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V9(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V10(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V11(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V12(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

Utility functions PHEV

In [74]:
def V13(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V14(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V15(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V16(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V17(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V18(Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

#### Probability function

In [12]:
def prob(mu, *args):
    total = 0.0
    for arg in args:
        total += np.exp(arg * mu)
    return [np.exp(mu*x) / total for x in args]

#### Simulation

In [81]:
def sim(probabilityVector):
    r = np.random.random()
    n = 0
    for i,value in enumerate(probabilityVector):
        n += value
        if r <= n:
            break
    return i

#### Mapping

In [78]:
mapping = {
    0:"ICEV_1",1:"ICEV_2",2:"ICEV_3",3:"ICEV_4",4:"ICEV_5",5:"ICEV_6",
    6:"BEV_1",7:"BEV_2",8:"BEV_3",9:"BEV_4",10:"BEV_5",11:"BEV_6",
    12:"PHEV_1",13:"PHEV_2",14:"PHEV_3",15:"PHEV_4",16:"PHEV_5",17:"PHEV_6"
}

#### Computing market share estimates

Calculating probabilities

In [75]:
mu = 1

probMatrix = []
for index, row in df_mod.iterrows():
    v1 = V1(np.random.random(),row['Purchaseprice_ICV_MINI'],row['Operationcost_ICV_MINI'],row['Yearlycost__ICV_MINI'],row['Range_ICV_MINI'],row['Acceleration_ICV_MINI'],row['Size_ICV1'],row['CO2_ICV_MINI'],row['Charginginfrastructure_ICV_MINI'],row['Chargingspeed_ICV_MINI'],row['Homedistnearestcharger_ICV_MINI'],row['Homechargeravailability_ICV_MINI'])
    v2 = V2(np.random.random(),row['Purchaseprice_ICV_LIL'],row['Operationcost_ICV_LIL'],row['Yearlycost__ICV_LIL'],row['Range_ICV_LIL'],row['Acceleration_ICV_LIL'],row['Size_ICV_LIL'],row['CO2_ICV_LIL'],row['Charginginfrastructure_ICV_LIL'],row['Chargingspeed_ICV_LIL'],row['Homedistnearestcharger_ICV_LIL'],row['Homechargeravailability_ICV_LIL'])
    v3 = V3(np.random.random(),row['Purchaseprice_ICV_MEL'],row['Operationcost_ICV_MEL'],row['Yearlycost__ICV_MEL'],row['Range_ICV_MEL'],row['Acceleration_ICV_MEL'],row['Size_ICV_MEL'],row['CO2_ICV_MEL'],row['Charginginfrastructure_ICV_MEL'],row['Chargingspeed_ICV_MEL'],row['Homedistnearestcharger_ICV_MEL'],row['Homechargeravailability_ICV_MEL'])
    v4 = V4(np.random.random(),row['Purchaseprice_ICV_STOR'],row['Operationcost_ICV_STOR'],row['Yearlycost__ICV_STOR'],row['Range_ICV_STOR'],row['Acceleration_ICV_STOR'],row['Size_ICV_STOR'],row['CO2_ICV_STOR'],row['Charginginfrastructure_ICV_STOR'],row['Chargingspeed_ICV_STOR'],row['Homedistnearestcharger_ICV_STOR'],row['Homechargeravailability_ICV_STOR'])
    v5 = V5(np.random.random(),row['Purchaseprice_ICV_PREM'],row['Operationcost_ICV_PREM'],row['Yearlycost__ICV_PREM'],row['Range_ICV_PREM'],row['Acceleration_ICV_PREM'],row['Size_ICV_PREM'],row['CO2_ICV_PREM'],row['Charginginfrastructure_ICV_PREM'],row['Chargingspeed_ICV_PREM'],row['Homedistnearestcharger_ICV_PREM'],row['Homechargeravailability_ICV_PREM'])
    v6 = V6(np.random.random(),row['Purchaseprice_ICV_LUK'],row['Operationcost_ICV_LUK'],row['Yearlycost__ICV_LUK'],row['Range_ICV_LUK'],row['Acceleration_ICV_LUK'],row['Size_ICV_LUK'],row['CO2_ICV_LUK'],row['Charginginfrastructure_ICV_LUK'],row['Chargingspeed_ICV_LUK'],row['Homedistnearestcharger_ICV_LUK'],row['Homechargeravailability_ICV_LUK'])

    v7 = V7(np.random.random(),row['Purchaseprice_BEV_MINI'],row['Operationcost_BEV_MINI'],row['Yearlycost__BEV_MINI'],row['Range_BEV_MINI'],row['Acceleration_BEV_MINI'],row['Size_BEV_MINI'],row['CO2_BEV_MINI'],row['Charginginfrastructure_BEV_MINI'],row['Chargingspeed_BEV_MINI'],row['Homedistnearestcharger_BEV_MINI'],row['Homechargeravailability_BEV_MINI'])
    v8 = V8(np.random.random(),row['Purchaseprice_BEV_LIL'],row['Operationcost_BEV_LIL'],row['Yearlycost__BEV_LIL'],row['Range_BEV_LIL'],row['Acceleration_BEV_LIL'],row['Size_BEV_LIL'],row['CO2_BEV_LIL'],row['Charginginfrastructure_BEV_LIL'],row['Chargingspeed_BEV_LIL'],row['Homedistnearestcharger_BEV_LIL'],row['Homechargeravailability_BEV_LIL'])
    v9 = V9(np.random.random(),row['Purchaseprice_BEV_MEL'],row['Operationcost_BEV_MEL'],row['Yearlycost__BEV_MEL'],row['Range_BEV_MEL'],row['Acceleration_BEV_MEL'],row['Size_BEV_MEL'],row['CO2_BEV_MEL'],row['Charginginfrastructure_BEV_MEL'],row['Chargingspeed_BEV_MEL'],row['Homedistnearestcharger_BEV_MEL'],row['Homechargeravailability_BEV_MEL'])
    v10 = V10(np.random.random(),row['Purchaseprice_BEV_STOR'],row['Operationcost_BEV_STOR'],row['Yearlycost__BEV_STOR'],row['Range_BEV_STOR'],row['Acceleration_BEV_STOR'],row['Size_BEV_STOR'],row['CO2_BEV_STOR'],row['Charginginfrastructure_BEV_STOR'],row['Chargingspeed_BEV_STOR'],row['Homedistnearestcharger_BEV_STOR'],row['Homechargeravailability_BEV_STOR'])
    v11 = V11(np.random.random(),row['Purchaseprice_BEV_PREM'],row['Operationcost_BEV_PREM'],row['Yearlycost__BEV_PREM'],row['Range_BEV_PREM'],row['Acceleration_BEV_PREM'],row['Size_BEV_PREM'],row['CO2_BEV_PREM'],row['Charginginfrastructure_BEV_PREM'],row['Chargingspeed_BEV_PREM'],row['Homedistnearestcharger_BEV_PREM'],row['Homechargeravailability_BEV_PREM'])
    v12 = V12(np.random.random(),row['Purchaseprice_BEV_LUK'],row['Operationcost_BEV_LUK'],row['Yearlycost__BEV_LUK'],row['Range_BEV_LUK'],row['Acceleration_BEV_LUK'],row['Size_BEV_LUK'],row['CO2_BEV_LUK'],row['Charginginfrastructure_BEV_LUK'],row['Chargingspeed_BEV_LUK'],row['Homedistnearestcharger_BEV_LUK'],row['Homechargeravailability_BEV_LUK'])

    v13 = V13(np.random.random(),row['Purchaseprice_PHEV_MINI'],row['Operationcost_PHEV_MINI'],row['Yearlycost__PHEV_MINI'],row['Range_PHEV_MINI'],row['Acceleration_PHEV_MINI'],row['Size_PHEV_MINI'],row['CO2_PHEV_MINI'],row['Charginginfrastructure_PHEV_MINI'],row['Chargingspeed_PHEV_MINI'],row['Homedistnearestcharger_PHEV_MINI'],row['Homechargeravailability_PHEV_MIN'])
    v14 = V14(np.random.random(),row['Purchaseprice_PHEV_LIL'],row['Operationcost_PHEV_LIL'],row['Yearlycost__PHEV_LIL'],row['Range_PHEV_LIL'],row['Acceleration_PHEV_LIL'],row['Size_PHEV_LIL'],row['CO2_PHEV_LIL'],row['Charginginfrastructure_PHEV_LIL'],row['Chargingspeed_PHEV_LIL'],row['Homedistnearestcharger_PHEV_LIL'],row['Homechargeravailability_PHEV_LIL'])
    v15 = V15(np.random.random(),row['Purchaseprice_PHEV_MEL'],row['Operationcost_PHEV_MEL'],row['Yearlycost__PHEV_MEL'],row['Range_PHEV_MEL'],row['Acceleration_PHEV_MEL'],row['Size_PHEV_MEL'],row['CO2_PHEV_MEL'],row['Charginginfrastructure_PHEV_MEL'],row['Chargingspeed_PHEV_MEL'],row['Homedistnearestcharger_PHEV_MEL'],row['Homechargeravailability_PHEV_MEL'])
    v16 = V16(np.random.random(),row['Purchaseprice_PHEV_STOR'],row['Operationcost_PHEV_STOR'],row['Yearlycost__PHEV_STOR'],row['Range_PHEV_STOR'],row['Acceleration_PHEV_STOR'],row['Size_PHEV_STOR'],row['CO2_PHEV_STOR'],row['Charginginfrastructure_PHEV_STOR'],row['Chargingspeed_PHEV_STOR'],row['Homedistnearestcharger_PHEV_STOR'],row['Homechargeravailability_PHEV_STO'])    
    v17 = V17(np.random.random(),row['Purchaseprice_PHEV_PREM'],row['Operationcost_PHEV_PREM'],row['Yearlycost__PHEV_PREM'],row['Range_PHEV_PREM'],row['Acceleration_PHEV_PREM'],row['Size_PHEV_PREM'],row['CO2_PHEV_PREM'],row['Charginginfrastructure_PHEV_PREM'],row['Chargingspeed_PHEV_PREM'],row['Homedistnearestcharger_PHEV_PREM'],row['Homechargeravailability_PHEV_PRE'])
    v18 = V18(row['Purchaseprice_PHEV_LUK'],row['Operationcost_PHEV_LUK'],row['Yearlycost__PHEV_LUK'],row['Range_PHEV_LUK'],row['Acceleration_PHEV_LUK'],row['Size_PHEV_LUK'],row['CO2_PHEV_LUK'],row['Charginginfrastructure_PHEV_LUK'],row['Chargingspeed_PHEV_LUK'],row['Homedistnearestcharger_PHEV_LUK'],row['Homechargeravailability_PHEV_LUK'])    
    
    
    probVector = prob(mu, v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18)
    probMatrix.append(probVector)

Sample enumeration

In [79]:
tot = len(probMatrix)
alt = len(mapping)
sampleEnum = {}
for i in range(len(mapping)):
    sampleEnum[mapping[i]] = round((sum([x[i] for x in probMatrix]) / tot) * 100,3)
pd.DataFrame([sampleEnum]).style.hide_index()

ICEV_1,ICEV_2,ICEV_3,ICEV_4,ICEV_5,ICEV_6,BEV_1,BEV_2,BEV_3,BEV_4,BEV_5,BEV_6,PHEV_1,PHEV_2,PHEV_3,PHEV_4,PHEV_5,PHEV_6
14.071000,5.275000,4.115000,3.988000,5.360000,5.956000,5.765000,4.874000,4.217000,4.188000,5.345000,5.954000,5.958000,5.386000,4.779000,5.149000,5.935000,3.685000


Simulation

In [109]:
r = []
for i in range(1000):
    x = [sim(x) for x in probMatrix]
    r.append({mapping[x[0]]:round((x[1]/ n)*100,3) for x in list(zip(Counter(x).keys(), Counter(x).values()))})
    
R = pd.DataFrame(r)

In [110]:
R.transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
ICEV_5,5.255,5.466,5.373,5.403,5.517,5.652,5.411,5.462,5.094,5.411,5.377,5.559,5.331,5.153,5.229,5.428,5.533,5.500,5.487,5.365,5.348,5.546,5.381,5.069,5.153,...,5.238,5.441,5.622,5.204,5.702,5.457,5.432,5.377,5.390,5.381,5.289,5.141,5.073,5.415,5.086,5.398,5.136,5.225,5.626,5.512,5.221,5.643,5.424,5.212,5.343
PHEV_2,5.411,5.479,5.310,5.673,5.238,5.479,5.318,5.419,5.462,5.166,5.327,5.255,5.352,5.170,5.457,5.381,5.381,5.318,5.022,5.601,5.297,5.403,5.686,5.483,5.500,...,5.681,5.441,5.276,5.242,5.550,5.276,5.394,5.187,5.170,5.398,5.217,5.314,5.728,5.212,5.352,5.487,5.322,5.293,5.572,5.272,5.280,5.512,5.369,5.250,5.491
PHEV_5,5.981,5.994,6.062,5.817,6.159,6.032,5.985,5.897,6.057,5.884,6.133,5.808,6.100,6.011,5.812,5.952,6.095,5.931,5.719,5.648,6.100,6.019,5.846,5.914,5.909,...,5.778,5.998,5.838,5.909,5.994,6.116,6.176,5.808,5.935,5.795,6.023,5.732,6.015,6.100,5.943,5.888,6.057,5.686,5.842,6.066,5.740,6.028,5.931,5.766,6.116
ICEV_1,14.159,13.762,14.721,13.931,14.007,14.184,14.134,14.201,14.273,14.163,14.113,14.049,14.180,13.669,14.193,13.998,13.986,13.931,14.252,14.408,13.969,13.770,14.467,14.349,14.412,...,14.134,13.969,14.028,14.053,13.779,14.345,13.990,13.880,14.049,14.311,14.049,14.100,14.087,14.353,14.205,13.939,14.286,13.952,14.281,14.315,14.252,14.134,14.231,14.163,14.011
BEV_1,5.931,5.740,5.462,5.939,5.618,5.728,5.787,5.601,5.369,5.821,5.855,5.605,5.833,5.859,5.766,5.728,5.871,5.757,5.791,5.664,5.504,5.508,5.618,5.909,5.880,...,5.715,5.597,5.821,5.901,5.800,5.905,5.576,5.884,5.694,5.690,5.740,5.745,5.829,5.563,6.057,5.935,5.817,5.757,5.791,5.736,5.842,5.728,5.985,5.719,5.833
BEV_5,5.259,5.124,5.419,5.263,5.153,5.365,5.529,5.107,5.174,5.115,5.331,5.415,5.377,5.381,5.314,5.441,5.297,5.470,5.462,5.466,5.390,5.369,5.153,5.293,5.339,...,5.449,5.305,5.369,5.453,5.022,5.179,5.491,5.500,5.605,5.360,5.441,5.322,5.681,5.331,5.407,5.424,5.225,5.174,5.343,5.204,5.204,5.141,5.453,5.407,5.411
PHEV_3,4.744,4.638,4.655,4.925,4.891,4.718,5.027,4.613,4.570,4.706,4.883,4.841,4.794,4.604,4.815,4.689,4.710,4.913,5.035,4.739,4.760,4.722,4.701,4.782,4.587,...,4.710,4.934,5.044,4.828,4.727,4.832,4.668,4.828,4.993,4.655,4.790,4.465,4.596,4.596,5.204,4.837,4.617,4.672,4.967,4.722,4.710,4.659,4.777,4.828,4.714
PHEV_4,5.086,5.120,5.136,5.145,5.314,4.866,5.369,5.035,5.517,4.951,5.500,4.832,5.082,5.065,5.048,4.967,5.022,4.972,5.145,5.077,5.365,5.158,5.107,5.204,5.141,...,5.031,5.086,5.111,5.301,4.934,5.136,4.980,5.229,5.162,5.128,5.234,5.225,5.229,5.107,4.989,5.124,5.229,5.301,4.980,5.424,5.158,5.048,5.056,5.360,5.200
BEV_6,5.990,6.188,5.901,5.778,6.083,5.952,5.943,6.049,5.888,5.905,5.724,5.795,5.994,5.969,5.960,6.230,5.681,6.057,5.918,6.100,5.884,6.062,6.011,5.990,5.580,...,5.964,6.019,5.981,5.876,6.116,5.863,5.918,6.176,6.184,5.791,6.154,5.998,6.002,5.855,6.036,6.015,6.095,5.935,5.884,5.842,5.893,5.867,6.087,5.800,5.846
PHEV_1,5.707,5.850,6.011,5.808,5.795,5.922,5.707,6.087,6.011,6.290,6.011,5.922,5.770,5.850,5.778,5.817,5.791,5.969,6.049,5.732,6.214,6.028,6.023,5.707,5.783,...,6.167,5.829,6.002,5.698,5.871,5.931,5.863,5.770,5.901,5.926,5.931,6.209,6.150,5.791,5.740,6.036,5.960,5.880,6.032,6.057,6.032,6.243,6.095,6.074,6.040


In [111]:
R.describe()

,ICEV_5,PHEV_2,PHEV_5,ICEV_1,BEV_1,BEV_5,PHEV_3,PHEV_4,BEV_6,PHEV_1,ICEV_2,PHEV_6,ICEV_3,BEV_2,BEV_4,BEV_3,ICEV_6,ICEV_4
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,5.361611,5.387055,5.930784,14.088011,5.759921,5.346056,4.774757,5.147305,5.955141,5.964001,5.270100,3.679275,4.115954,4.867624,4.182281,4.224965,5.957061,3.988055
std,0.140655,0.146931,0.153092,0.221218,0.148015,0.143213,0.134298,0.142998,0.151142,0.151231,0.149323,0.120229,0.128445,0.138157,0.123361,0.127799,0.153502,0.124084
min,4.845000,4.951000,5.474000,13.428000,5.352000,4.891000,4.397000,4.672000,5.457000,5.491000,4.769000,3.291000,3.709000,4.444000,3.802000,3.878000,5.576000,3.654000
25%,5.267000,5.284000,5.829000,13.939000,5.652000,5.250000,4.689000,5.048000,5.855000,5.855000,5.170000,3.602000,4.026000,4.769000,4.097000,4.135000,5.846000,3.899000
50%,5.360000,5.386000,5.931000,14.087000,5.764000,5.352000,4.777000,5.145000,5.952000,5.969000,5.272000,3.675000,4.114000,4.870000,4.186000,4.228000,5.960000,3.987000
75%,5.457000,5.487000,6.036000,14.240000,5.863000,5.446000,4.866000,5.242000,6.054000,6.062000,5.373000,3.759000,4.203000,4.959000,4.263000,4.313000,6.057000,4.072000
max,5.795000,5.855000,6.353000,14.721000,6.268000,5.850000,5.204000,5.597000,6.383000,6.475000,5.804000,4.042000,4.537000,5.267000,4.655000,4.621000,6.454000,4.520000


Observed market shares

In [153]:
df_test = df_mod.copy(deep=True)
df_test['count'] = 0
df_test = df_test.groupby('CHOICE18', as_index=False)['count'].count()
df_test.transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
CHOICE18,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
count,1538,2184,3604,1574,864,187,1254,1583,4118,2177,660,137,543,670,1713,670,150,48


In [154]:
df_test['share'] = df_test['count'] / n * 100
df_test[['CHOICE18','share']].transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
CHOICE18,1.000000,2.00000,3.000000,4.000000,5.000000,6.000000,7.00000,8.00000,9.00000,10.000000,11.000000,12.000000,13.000000,14.000000,15.000000,16.000000,17.000000,18.000000
share,6.496579,9.22531,15.223452,6.648644,3.649573,0.789896,5.29695,6.68666,17.39461,9.195742,2.787869,0.578694,2.293655,2.830109,7.235786,2.830109,0.633606,0.202754


#### Scenario 1
Change in cost and range of BEV/PHEV (better technologies, scaling-up production, subsidies).

In [155]:
df_s1 = df_mod.copy(deep=True)

In [158]:
col = df_s1.columns.to_list()
col_bev = []
col_phev = []
for c in col:
    if Search("Purchaseprice_BEV_",c):
        col_bev.append(c)
    if Search("Purchaseprice_PHEV_",c):
        col_phev.append(c)
print(col_bev)
print(col_phev)

['Purchaseprice_BEV_MINI', 'Purchaseprice_BEV_LIL', 'Purchaseprice_BEV_MEL', 'Purchaseprice_BEV_STOR', 'Purchaseprice_BEV_PREM', 'Purchaseprice_BEV_LUK']
['Purchaseprice_PHEV_MINI', 'Purchaseprice_PHEV_LIL', 'Purchaseprice_PHEV_MEL', 'Purchaseprice_PHEV_STOR', 'Purchaseprice_PHEV_PREM', 'Purchaseprice_PHEV_LUK']


In [162]:
for c in col_bev:
    df_s1[c+"_NEW"] = df_s1[c] * 0.6

df_s1[df_s1['Purchaseprice_BEV_MINI'] != 0][['Purchaseprice_BEV_MINI','Purchaseprice_BEV_MINI_NEW']].head(2)

,Purchaseprice_BEV_MINI,Purchaseprice_BEV_MINI_NEW
24,125000,75000.0
25,125000,75000.0


In [165]:
for c in col_phev:
    df_s1[c+"_NEW"] = df_s1[c] * 0.8

df_s1[df_s1['Purchaseprice_PHEV_MINI'] != 0][['Purchaseprice_PHEV_MINI','Purchaseprice_PHEV_MINI_NEW']].head(2)

,Purchaseprice_PHEV_MINI,Purchaseprice_PHEV_MINI_NEW
28,135000,108000.0
29,135000,108000.0


Calculating probabilities

In [168]:
mu = 1

probMatrix_s1 = []
for index, row in df_s1.iterrows():
    v1 = V1(np.random.random(),row['Purchaseprice_ICV_MINI'],row['Operationcost_ICV_MINI'],row['Yearlycost__ICV_MINI'],row['Range_ICV_MINI'],row['Acceleration_ICV_MINI'],row['Size_ICV1'],row['CO2_ICV_MINI'],row['Charginginfrastructure_ICV_MINI'],row['Chargingspeed_ICV_MINI'],row['Homedistnearestcharger_ICV_MINI'],row['Homechargeravailability_ICV_MINI'])
    v2 = V2(np.random.random(),row['Purchaseprice_ICV_LIL'],row['Operationcost_ICV_LIL'],row['Yearlycost__ICV_LIL'],row['Range_ICV_LIL'],row['Acceleration_ICV_LIL'],row['Size_ICV_LIL'],row['CO2_ICV_LIL'],row['Charginginfrastructure_ICV_LIL'],row['Chargingspeed_ICV_LIL'],row['Homedistnearestcharger_ICV_LIL'],row['Homechargeravailability_ICV_LIL'])
    v3 = V3(np.random.random(),row['Purchaseprice_ICV_MEL'],row['Operationcost_ICV_MEL'],row['Yearlycost__ICV_MEL'],row['Range_ICV_MEL'],row['Acceleration_ICV_MEL'],row['Size_ICV_MEL'],row['CO2_ICV_MEL'],row['Charginginfrastructure_ICV_MEL'],row['Chargingspeed_ICV_MEL'],row['Homedistnearestcharger_ICV_MEL'],row['Homechargeravailability_ICV_MEL'])
    v4 = V4(np.random.random(),row['Purchaseprice_ICV_STOR'],row['Operationcost_ICV_STOR'],row['Yearlycost__ICV_STOR'],row['Range_ICV_STOR'],row['Acceleration_ICV_STOR'],row['Size_ICV_STOR'],row['CO2_ICV_STOR'],row['Charginginfrastructure_ICV_STOR'],row['Chargingspeed_ICV_STOR'],row['Homedistnearestcharger_ICV_STOR'],row['Homechargeravailability_ICV_STOR'])
    v5 = V5(np.random.random(),row['Purchaseprice_ICV_PREM'],row['Operationcost_ICV_PREM'],row['Yearlycost__ICV_PREM'],row['Range_ICV_PREM'],row['Acceleration_ICV_PREM'],row['Size_ICV_PREM'],row['CO2_ICV_PREM'],row['Charginginfrastructure_ICV_PREM'],row['Chargingspeed_ICV_PREM'],row['Homedistnearestcharger_ICV_PREM'],row['Homechargeravailability_ICV_PREM'])
    v6 = V6(np.random.random(),row['Purchaseprice_ICV_LUK'],row['Operationcost_ICV_LUK'],row['Yearlycost__ICV_LUK'],row['Range_ICV_LUK'],row['Acceleration_ICV_LUK'],row['Size_ICV_LUK'],row['CO2_ICV_LUK'],row['Charginginfrastructure_ICV_LUK'],row['Chargingspeed_ICV_LUK'],row['Homedistnearestcharger_ICV_LUK'],row['Homechargeravailability_ICV_LUK'])

    v7 = V7(np.random.random(),row['Purchaseprice_BEV_MINI_NEW'],row['Operationcost_BEV_MINI'],row['Yearlycost__BEV_MINI'],row['Range_BEV_MINI'],row['Acceleration_BEV_MINI'],row['Size_BEV_MINI'],row['CO2_BEV_MINI'],row['Charginginfrastructure_BEV_MINI'],row['Chargingspeed_BEV_MINI'],row['Homedistnearestcharger_BEV_MINI'],row['Homechargeravailability_BEV_MINI'])
    v8 = V8(np.random.random(),row['Purchaseprice_BEV_LIL_NEW'],row['Operationcost_BEV_LIL'],row['Yearlycost__BEV_LIL'],row['Range_BEV_LIL'],row['Acceleration_BEV_LIL'],row['Size_BEV_LIL'],row['CO2_BEV_LIL'],row['Charginginfrastructure_BEV_LIL'],row['Chargingspeed_BEV_LIL'],row['Homedistnearestcharger_BEV_LIL'],row['Homechargeravailability_BEV_LIL'])
    v9 = V9(np.random.random(),row['Purchaseprice_BEV_MEL_NEW'],row['Operationcost_BEV_MEL'],row['Yearlycost__BEV_MEL'],row['Range_BEV_MEL'],row['Acceleration_BEV_MEL'],row['Size_BEV_MEL'],row['CO2_BEV_MEL'],row['Charginginfrastructure_BEV_MEL'],row['Chargingspeed_BEV_MEL'],row['Homedistnearestcharger_BEV_MEL'],row['Homechargeravailability_BEV_MEL'])
    v10 = V10(np.random.random(),row['Purchaseprice_BEV_STOR_NEW'],row['Operationcost_BEV_STOR'],row['Yearlycost__BEV_STOR'],row['Range_BEV_STOR'],row['Acceleration_BEV_STOR'],row['Size_BEV_STOR'],row['CO2_BEV_STOR'],row['Charginginfrastructure_BEV_STOR'],row['Chargingspeed_BEV_STOR'],row['Homedistnearestcharger_BEV_STOR'],row['Homechargeravailability_BEV_STOR'])
    v11 = V11(np.random.random(),row['Purchaseprice_BEV_PREM_NEW'],row['Operationcost_BEV_PREM'],row['Yearlycost__BEV_PREM'],row['Range_BEV_PREM'],row['Acceleration_BEV_PREM'],row['Size_BEV_PREM'],row['CO2_BEV_PREM'],row['Charginginfrastructure_BEV_PREM'],row['Chargingspeed_BEV_PREM'],row['Homedistnearestcharger_BEV_PREM'],row['Homechargeravailability_BEV_PREM'])
    v12 = V12(np.random.random(),row['Purchaseprice_BEV_LUK_NEW'],row['Operationcost_BEV_LUK'],row['Yearlycost__BEV_LUK'],row['Range_BEV_LUK'],row['Acceleration_BEV_LUK'],row['Size_BEV_LUK'],row['CO2_BEV_LUK'],row['Charginginfrastructure_BEV_LUK'],row['Chargingspeed_BEV_LUK'],row['Homedistnearestcharger_BEV_LUK'],row['Homechargeravailability_BEV_LUK'])

    v13 = V13(np.random.random(),row['Purchaseprice_PHEV_MINI_NEW'],row['Operationcost_PHEV_MINI'],row['Yearlycost__PHEV_MINI'],row['Range_PHEV_MINI'],row['Acceleration_PHEV_MINI'],row['Size_PHEV_MINI'],row['CO2_PHEV_MINI'],row['Charginginfrastructure_PHEV_MINI'],row['Chargingspeed_PHEV_MINI'],row['Homedistnearestcharger_PHEV_MINI'],row['Homechargeravailability_PHEV_MIN'])
    v14 = V14(np.random.random(),row['Purchaseprice_PHEV_LIL_NEW'],row['Operationcost_PHEV_LIL'],row['Yearlycost__PHEV_LIL'],row['Range_PHEV_LIL'],row['Acceleration_PHEV_LIL'],row['Size_PHEV_LIL'],row['CO2_PHEV_LIL'],row['Charginginfrastructure_PHEV_LIL'],row['Chargingspeed_PHEV_LIL'],row['Homedistnearestcharger_PHEV_LIL'],row['Homechargeravailability_PHEV_LIL'])
    v15 = V15(np.random.random(),row['Purchaseprice_PHEV_MEL_NEW'],row['Operationcost_PHEV_MEL'],row['Yearlycost__PHEV_MEL'],row['Range_PHEV_MEL'],row['Acceleration_PHEV_MEL'],row['Size_PHEV_MEL'],row['CO2_PHEV_MEL'],row['Charginginfrastructure_PHEV_MEL'],row['Chargingspeed_PHEV_MEL'],row['Homedistnearestcharger_PHEV_MEL'],row['Homechargeravailability_PHEV_MEL'])
    v16 = V16(np.random.random(),row['Purchaseprice_PHEV_STOR_NEW'],row['Operationcost_PHEV_STOR'],row['Yearlycost__PHEV_STOR'],row['Range_PHEV_STOR'],row['Acceleration_PHEV_STOR'],row['Size_PHEV_STOR'],row['CO2_PHEV_STOR'],row['Charginginfrastructure_PHEV_STOR'],row['Chargingspeed_PHEV_STOR'],row['Homedistnearestcharger_PHEV_STOR'],row['Homechargeravailability_PHEV_STO'])    
    v17 = V17(np.random.random(),row['Purchaseprice_PHEV_PREM_NEW'],row['Operationcost_PHEV_PREM'],row['Yearlycost__PHEV_PREM'],row['Range_PHEV_PREM'],row['Acceleration_PHEV_PREM'],row['Size_PHEV_PREM'],row['CO2_PHEV_PREM'],row['Charginginfrastructure_PHEV_PREM'],row['Chargingspeed_PHEV_PREM'],row['Homedistnearestcharger_PHEV_PREM'],row['Homechargeravailability_PHEV_PRE'])
    v18 = V18(row['Purchaseprice_PHEV_LUK_NEW'],row['Operationcost_PHEV_LUK'],row['Yearlycost__PHEV_LUK'],row['Range_PHEV_LUK'],row['Acceleration_PHEV_LUK'],row['Size_PHEV_LUK'],row['CO2_PHEV_LUK'],row['Charginginfrastructure_PHEV_LUK'],row['Chargingspeed_PHEV_LUK'],row['Homedistnearestcharger_PHEV_LUK'],row['Homechargeravailability_PHEV_LUK'])    
    
    
    probVector_s1 = prob(mu, v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18)
    probMatrix_s1.append(probVector_s1)

Sample enumeration

In [169]:
tot = len(probMatrix_s1)
alt = len(mapping)
sampleEnum_s1 = {}
for i in range(len(mapping)):
    sampleEnum_s1[mapping[i]] = round((sum([x[i] for x in probMatrix_s1]) / tot) * 100,3)
pd.DataFrame([sampleEnum_s1]).style.hide_index()

ICEV_1,ICEV_2,ICEV_3,ICEV_4,ICEV_5,ICEV_6,BEV_1,BEV_2,BEV_3,BEV_4,BEV_5,BEV_6,PHEV_1,PHEV_2,PHEV_3,PHEV_4,PHEV_5,PHEV_6
13.628000,5.104000,4.002000,3.881000,5.149000,5.737000,5.843000,5.350000,5.419000,4.813000,5.317000,5.743000,5.763000,5.228000,4.713000,5.001000,5.750000,3.559000


#### Scenario 2
Better availability of charging stations (investments in infrastructure, subsidies).

#### Scenario 3
Evolution of oil and electricity prices (EU’s climate policy.: electrification, bigger share of intermittent sources, etc.) 